# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [40]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [41]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")
humidity = city_data_df["Humidity"].astype(int)
# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,summerside,46.3959,-63.7876,14.50,77,13,1.54,CA,1692750014
1,1,kundiawa,-6.0167,144.9667,21.56,75,78,1.73,PG,1692749736
2,2,al bawiti,28.3492,28.8659,27.18,35,0,2.39,EG,1692750015
3,3,bikenibeu village,1.3673,173.1241,28.70,73,75,3.34,KI,1692750015
4,4,blackmans bay,-43.0167,147.3167,13.07,69,62,4.03,AU,1692750015


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [43]:
%%capture --no-display
# Configure the map plot
plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 25) & (city_data_df["Max Temp"] > 15)]

city_data_df = city_data_df.loc[(city_data_df["Humidity"] < 85) & (city_data_df["Humidity"] > 50)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,kundiawa,-6.0167,144.9667,21.56,75,78,1.73,PG,1692749736
7,7,tsiombe,-25.3000,45.4833,20.75,66,0,2.71,MG,1692750015
11,11,morden,49.1919,-98.1014,20.24,83,97,2.06,CA,1692749997
15,15,adamstown,-25.0660,-130.1015,18.84,67,100,12.24,PN,1692750017
16,16,yellowknife,62.4560,-114.3525,20.29,58,75,4.12,CA,1692749896
...,...,...,...,...,...,...,...,...,...,...
569,569,kone,-21.0595,164.8658,23.66,52,18,1.71,NC,1692750113
570,570,huron,41.1501,-82.5499,20.93,81,41,2.83,US,1692750113
573,573,wajir,1.7471,40.0573,24.16,62,24,6.57,KE,1692749825
574,574,levuka,50.1834,-105.4011,22.85,73,75,4.12,CA,1692750114


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = city_data_df[{"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity}]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

C:\Users\stara\AppData\Local\Temp\ipykernel_18840\2468048700.py:8: FutureWarning: Passing a dict as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = city_data_df[{"City": city,
C:\Users\stara\AppData\Local\Temp\ipykernel_18840\2468048700.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Country,Lat,Lng,Humidity,Hotel Name
1,kundiawa,PG,-6.0167,144.9667,75,
7,tsiombe,MG,-25.3000,45.4833,66,
11,morden,CA,49.1919,-98.1014,83,
15,adamstown,PN,-25.0660,-130.1015,67,
16,yellowknife,CA,62.4560,-114.3525,58,
...,...,...,...,...,...,...
569,kone,NC,-21.0595,164.8658,52,
570,huron,US,41.1501,-82.5499,81,
573,wajir,KE,1.7471,40.0573,62,
574,levuka,CA,50.1834,-105.4011,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 500
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kundiawa - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
morden - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
yellowknife - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
college - nearest hotel: No hotel found
sitka - nearest hotel: Aspen Suites Hotel
pamiers - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
albany - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
rifle - nearest hotel: No hotel found
price - nearest hotel: No hotel found
puerto baquerizo moreno - nearest hotel: No hotel found
nouadhibou - nearest hotel: No hotel found
rosetta - nearest hotel: No hotel found
takestan - nearest hotel: No hotel found
taltal - nearest hotel: No hotel found
bodaybo - nearest hotel: НОЙ
klyuchi - nearest hotel: No hotel found
urangan - nearest hotel: No hotel found
nuku'alofa - nearest hotel: City Hotel
vilyuchinsk - nearest hotel: Победа
ki

,City,Country,Lat,Lng,Humidity,Hotel Name
1,kundiawa,PG,-6.0167,144.9667,75,No hotel found
7,tsiombe,MG,-25.3000,45.4833,66,No hotel found
11,morden,CA,49.1919,-98.1014,83,No hotel found
15,adamstown,PN,-25.0660,-130.1015,67,No hotel found
16,yellowknife,CA,62.4560,-114.3525,58,No hotel found
...,...,...,...,...,...,...
569,kone,NC,-21.0595,164.8658,52,Pacifik Appartels
570,huron,US,41.1501,-82.5499,81,No hotel found
573,wajir,KE,1.7471,40.0573,62,Kilimanjaro
574,levuka,CA,50.1834,-105.4011,73,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)